# Real-Time Credit Card Fraud Detection

## Loading the File and Performing Basic Data Type Conversions

In [0]:
# import modules from pyspark
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SQLContext
import pandas as pd

# uncomment the following line if running pyspark from the notebook itself
# spark = SparkSession.builder.enableHiveSupport().getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
sqlContext = SQLContext(spark)

# Read The data
cc_raw = (spark.read
  .option("header", "true")
  .csv("fraudTrain.csv"))

In [0]:
# Read The data
cc_raw = (spark.read
  .option("header", "true")
  .csv("s3://group9-ml-project/fraudTrain.csv"))

In [0]:
# Register spark SQL tables

cc_raw.createOrReplaceTempView("cc_raw")

In [0]:
cc_raw.describe().toPandas()

In [0]:
spark.sql("DROP TABLE IF EXISTS cc")

In [0]:
spark.sql(""" CREATE TABLE IF NOT EXISTS cc AS 
                SELECT timestamp(trans_date_trans_time) as trans_date_trans_time,
                       cc_num, 
                       merchant, 
                       category,
                       double(amt) as amt,
                       first,
                       last,
                       gender,
                       street,
                       city,
                       state,
                       zip,
                       double(lat) as lat,
                       double(long) as long,
                       double(city_pop) as city_pop,
                       job,
                       date(dob) as dob,
                       trans_num,
                       unix_time,
                       double(merch_lat) as merch_lat,
                       double(merch_long) as merch_long,
                       int(is_fraud) as is_fraud
                  FROM cc_raw
          """)

In [0]:
sqlContext.cacheTable("cc")

cc = spark.sql("SELECT * FROM cc")

## Exploratory Data Analysis

### A Look at the Data and its Basic Statistics

In [0]:
cc.columns

In [0]:
import pandas as pd

pd.DataFrame(cc.take(10), columns=cc.columns)

In [0]:
# making sure there are no missing values
cc.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in cc.columns)).toPandas()

### Visualizing the Data Distribution

In [0]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(4 , 2, figsize=(15, 20))
fig.suptitle('CC Fraud Data Distribution')

for idx, column in enumerate(['amt', 'city_pop', 'lat', 'long', 'merch_lat', 'merch_long', 'unix_time', 'is_fraud']):
    # Show histogram of the column
    bins, counts = cc.select(column).rdd.flatMap(lambda x: x).map(float).histogram(20)
    axs[idx//2][idx%2].set_title(column)
    axs[idx//2][idx%2].hist(bins[:-1], bins=bins, weights=counts)
    
plt.show()

In [0]:
cc.select("category").groupby("category").count().toPandas()

In [0]:
cc.select("gender").groupby("gender").count().toPandas()

In [0]:
# diamonds.select("clarity").groupby("clarity").count().toPandas()

### Findings Summary and Strategy for Data Preparation

The data appears to be clean with no missing values. However, given that carat and price are heavily right skewed, I will apply a logarithmic transformation to them. The categorical features are of String type. As such, I will first need to encode them to a numeric indices and then one-hot encode the indices.

## Data Preparation

### Data Transformation

In [0]:
from pyspark.ml.feature import StringIndexer

# convert string categorical features to numeric indices
inputs = ['merchant', 'category', 'gender', 'city', 'state', 'job']
outputs = ['merchant_idx', 'category_idx', 'gender_idx', 'city_idx', 'state_idx', 'job_idx']
stringIndexer = StringIndexer(inputCols=inputs, outputCols=outputs)
model = stringIndexer.fit(cc)
cc_idx = model.transform(cc)

In [0]:
pd.DataFrame(cc_idx.take(50), columns=cc_idx.columns)

In [0]:
cc.count()

In [0]:
from pyspark.ml.feature import OneHotEncoder

# one-hot encode indexed categorical features
inputs_1hot = ['merchant_idx', 'category_idx', 'city_idx', 'state_idx', 'job_idx']
outputs_1hot = ['merchant_1hot', 'category_1hot', 'city_1hot', 'state_1hot', 'job_1hot']

oneHotEncoder = OneHotEncoder(inputCols=inputs_1hot, outputCols=outputs_1hot)
model_1hot = oneHotEncoder.fit(cc_idx)
cc_1hot = model_1hot.transform(cc_idx)

In [0]:
pd.DataFrame(cc_1hot.take(50), columns=cc_1hot.columns)

In [0]:
# apply logarithmic transformation to carat and price features
cc_prepped = cc_1hot.withColumn('amt_log', log(col("amt"))) \
                                .withColumn('city_pop_log', log(col("city_pop")))

In [0]:
pd.DataFrame(cc_prepped.take(50), columns=cc_prepped.columns)

### Review of Revised Distribution for Amt and City_Pop

In [0]:
# plot revised data distribution
fig, axs = plt.subplots(1 , 2, figsize=(15, 5))
fig.suptitle('Prepped CC Data Distribution')

for idx, column in enumerate(['amt_log', 'city_pop_log']):
    # Show histogram of the column
    bins, counts = cc_prepped.select(column).rdd.flatMap(lambda x: x).map(float).histogram(20)
    axs[idx%2].set_title(column)
    axs[idx%2].hist(bins[:-1], bins=bins, weights=counts)
    
plt.show()

### Feature Extraction and Training/Test DataSet Creation

In [0]:
cc_prepped.columns

In [0]:
from pyspark.ml.feature import VectorAssembler

# assemble the prepped features into one single vector.
featureCols = ['amt_log', 'city_pop_log', 'job_1hot', 'state_1hot', 'category_1hot', 'gender_idx']
assembler = (VectorAssembler()
  .setInputCols(featureCols)
  .setOutputCol("features"))

cc_final = assembler.transform(cc_prepped)

In [0]:
# diamonds_final.toPandas()

In [0]:
# setup training and test datasets

training, test = cc_final.randomSplit([0.7, 0.3])

#  Going to cache the data to make sure things stay snappy!
# training.cache()
# test.cache()

print(training.count()) # Why execute count here??
print(test.count())

In [0]:
test.select("is_fraud").groupby("is_fraud").count().toPandas()

## ML Training and Prediction - RandomForestClassifier

In [0]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(numTrees=3, maxDepth=2, labelCol="is_fraud", seed=42,
    leafCol="leafId")
rf.setFeaturesCol("features")

model = rf.fit(training)

In [0]:
preds = model.transform(test) 

In [0]:
preds.columns

In [0]:
pd.DataFrame(preds.take(50), columns=preds.columns)

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Instantiate the evaluator
bce= BinaryClassificationEvaluator(rawPredictionCol= "rawPrediction",
                                   labelCol="is_fraud", 
                                   metricName= "areaUnderROC")
                                   
bce.evaluate(preds)

In [0]:
from pyspark.mllib.evaluation import MulticlassMetrics

preds_float = preds \
    .select("prediction", "is_fraud") \
    .withColumn("is_fraud", col("is_fraud").cast(DoubleType())) \
    .orderBy("prediction")

preds_float.take(50)

cm = MulticlassMetrics(preds_float.rdd.map(tuple))

# print(cm.confusionMatrix().toArray())

#show the confusion matrix as a pandas df for clearer presentation
pd.DataFrame(cm.confusionMatrix().toArray(),
             columns= ["true positive", "true negative"],
             index= ["predicted positive", "predicted negative"])

In [0]:
preds.select("is_fraud").groupby("is_fraud").count().toPandas()

In [0]:
preds.select("prediction").groupby("prediction").count().toPandas()

### Results Analysis

Recognizing that I am working with logarithmic values for the target variable, I found this first iteration to yield low error rates and high accuracy an R Squared metric of .975

## ML Training and Prediction - XGBoost

In [0]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml import Pipeline

from xgboost.spark import SparkXGBRegressor

xgbModel = SparkXGBRegressor(
  features_col="features",
  label_col="price_log",
  num_workers=4,
)

paramGrid_xgb = (ParamGridBuilder()
  .addGrid(xgbModel.max_depth, [3, 5, 7])
  .addGrid(xgbModel.learning_rate, [0.1, 0.2])
  .addGrid(xgbModel.n_estimators, [300, 400])
  .build())

stages_xgb = [xgbModel]

pipeline_xgb = Pipeline().setStages(stages_xgb)

cv_xgb = (CrossValidator() # you can feel free to change the number of folds used in cross validation as well
  .setEstimator(pipeline_xgb) # the estimator can also just be an individual model rather than a pipeline
  .setEstimatorParamMaps(paramGrid_xgb)
  .setEvaluator(RegressionEvaluator().setLabelCol("price_log")))

pipelineFitted_xgb = cv_xgb.fit(training)

In [0]:
holdout_xgb = (pipelineFitted_xgb.bestModel
  .transform(test))
  
holdout_xgb.toPandas()

In [0]:
from pyspark.mllib.evaluation import RegressionMetrics

rm_xgb = RegressionMetrics(holdout_xgb.select("prediction", "price_log").rdd.map(lambda x:  (x[0], x[1])))

print("MSE: ", rm_xgb.meanSquaredError)
print("MAE: ", rm_xgb.meanAbsoluteError)
print("RMSE Squared: ", rm_xgb.rootMeanSquaredError)
print("R Squared: ", rm_xgb.r2)
print("Explained Variance: ", rm_xgb.explainedVariance, "\n")

### Results Analysis

It was interesting to compare the Random Forest algorithm against XGBoost. With the grid parameters tested, XGBoost outperformed Random Forest in virtually all metrics, with an R Squared score of 0.992 compared to 0.975 for Random Forest.